In [ ]:
import pandas as pd
import numpy as np

# --- KONFIGURASI ---
input_path = '2025 data cabang alat berat.xlsx'
output_path = 'Summary_Cabang_Alat_Berat_2025_v2.xlsx'

# 1. Daftar Nama Cabang Lengkap Awal (46 Cabang)
full_branch_names = [
    "AMBON", "ARAR", "BAU BAU", "BIAK", "BITUNG", "BENGKULU", "BUNGKU", "BATULICIN", "BELAWAN", 
    "BANJARMASIN", "BOEPINANG", "BALIKPAPAN", "BERAU", "BANDA ACEH", "BATAM", "BUATAN", "DOBO", 
    "FAK FAK", "GORONTALO", "JAKARTA", "JAYAPURA", "KAIMANA", "KENDARI", "KETAPANG", "KUALA TANJUNG", 
    "MERAUKE", "MAKASSAR", "MANOKWARI", "NABIRE", "NUNUKAN", "PALU", "PADANG", "PALEMBANG", 
    "PONTIANAK", "PERAWANG", "SURABAYA", "SAMARINDA", "SEMARANG", "SAMPIT", "SORONG", "SERUI", 
    "TANGKIANG", "TIMIKA", "TARAKAN", "TERNATE", "TUAL"
]

# Tambahkan cabang baru ke daftar proses
additional_branches = ["MERAK", "MARUNI", "OKI"]
processing_branch_list = full_branch_names + additional_branches

# 2. Daftar Status yang Diambil
target_statuses = ['FXD', 'FAC', 'STR', 'MTA', 'MAS', 'MTB', 'FTL', 'FOB', 'MXD', 'MTL', 'MOB', 'FIT', 'MIT']

# 3. Mapping Ukuran Container (Update: 10 CO/DC -> 20 Feet)
size_map = {
    # 20 Feet Types
    '20 CO': '20 Feet', '20 DC': '20 Feet', '20 RH': '20 Feet', '20 RM': '20 Feet', '21 DC': '20 Feet',
    '20 FT': '20 Feet', '20 HC': '20 Feet', '20 IT': '20 Feet', '20 OT': '20 Feet', '20 RF': '20 Feet',
    '10 CO': '20 Feet', '10 DC': '20 Feet', # Added per request
    # 40 Feet Types
    '40 FT': '40 Feet', '40 HC': '40 Feet', '40 RH': '40 Feet', '40 RM': '40 Feet',
    '40 CO': '40 Feet', '40 DC': '40 Feet', '40 IT': '40 Feet', '40 OT': '40 Feet', '40 RF': '40 Feet',
    '41 DC': '40 Feet', '45 HC': '40 Feet'
}

# 4. Mapping Bulan
month_map = {
    'Jan': 'Januari', 'Feb': 'Februari', 'Mar': 'Maret', 'Apr': 'April', 'May': 'Mei', 'Jun': 'Juni',
    'Jul': 'Juli', 'Aug': 'Agustus', 'Sep': 'September', 'Oct': 'Oktober', 'Nov': 'November', 'Dec': 'Desember'
}
month_order = list(month_map.values())

# --- VARIABLES FOR LOGGING ---
skipped_branches = set()
skipped_container_types = set()

# --- PROSES MEMBACA DATA ---

print("Membaca file Excel...")
df_full = pd.read_excel(input_path, header=None)

# Cari lokasi baris penanda 'CURRSTATE' dan 'Row Labels'
currstate_rows = df_full[df_full[0] == 'CURRSTATE'].index.tolist()
row_labels_rows = df_full[df_full[0] == 'Row Labels'].index.tolist()

# -- MEMBUAT MAPPING CABANG --
if not row_labels_rows:
    raise ValueError("Tidak ditemukan baris 'Row Labels' dalam file Excel.")

first_header = row_labels_rows[0]
block1_branches = df_full.iloc[first_header+1 : first_header+1+46, 0].tolist()

# 1. Mapping dari Blok Pertama (46 Cabang)
branch_map = dict(zip(block1_branches, full_branch_names))

# 2. Menambahkan Mapping Manual untuk Cabang Tambahan
manual_map = {
    'MRK': 'MERAK',
    'MRN': 'MARUNI',
    'OKI': 'OKI'
}
branch_map.update(manual_map)

# Buat Reverse Map untuk lookup singkatan nanti (Nama Lengkap -> Singkatan)
# Priority: Manual map overrides if duplicates exist (unlikely here)
full_to_abbr = {v: k for k, v in branch_map.items()}

all_data = []

print("Memproses blok data...")
for i, start_idx in enumerate(currstate_rows):
    status = df_full.iloc[start_idx, 1]
    
    if status not in target_statuses:
        continue
        
    header_idx = next((r for r in row_labels_rows if r > start_idx), None)
    if header_idx is None: continue
    
    next_start = currstate_rows[i+1] if i+1 < len(currstate_rows) else len(df_full)
    
    ownership_row = df_full.iloc[header_idx - 2]
    month_row = df_full.iloc[header_idx - 1]
    type_row = df_full.iloc[header_idx]
    
    col_map = {}
    curr_own = None
    curr_mon = None
    
    for c in range(1, df_full.shape[1]):
        if pd.notna(ownership_row[c]): curr_own = ownership_row[c]
        if pd.notna(month_row[c]): curr_mon = month_row[c]
        
        ctype = type_row[c]
        
        if pd.notna(ctype):
            col_map[c] = {
                'ownership': curr_own,
                'month': curr_mon,
                'type': ctype
            }
            
    for r in range(header_idx + 1, next_start):
        row_label = df_full.iloc[r, 0]
        
        if pd.isna(row_label) or str(row_label).lower() == 'grand total':
            continue
        
        # Cek Cabang
        if row_label in branch_map:
            branch_name = branch_map[row_label]
        elif row_label in processing_branch_list: # Jika sudah nama lengkap
            branch_name = row_label
        else:
            skipped_branches.add(row_label)
            continue 
            
        for c, meta in col_map.items():
            val = df_full.iloc[r, c]
            if pd.notna(val) and val != 0:
                all_data.append({
                    'Cabang': branch_name,
                    'Status': status,
                    'Bulan': meta['month'],
                    'Type': meta['type'],
                    'Count': val
                })

# --- LAPORAN DATA YANG DI-SKIP ---
print("\n" + "="*60)
print("LAPORAN DATA YANG TIDAK MASUK PERHITUNGAN (DIABAIKAN)")
print("="*60)

if skipped_branches:
    print(f"\n[CABANG] Ditemukan {len(skipped_branches)} nama cabang yang tidak dikenali:")
    for b in sorted(list(skipped_branches)):
        print(f" - {b}")
else:
    print("\n[CABANG] Semua cabang berhasil dikenali.")

# --- PENGOLAHAN DATA ---

if not all_data:
    print("\n[ERROR] Tidak ada data valid yang ditemukan untuk diproses.")
else:
    df_raw = pd.DataFrame(all_data)

    unique_types_in_data = df_raw['Type'].unique()
    for t in unique_types_in_data:
        if t not in size_map:
            skipped_container_types.add(t)

    if skipped_container_types:
        print(f"\n[CONTAINER] Ditemukan {len(skipped_container_types)} tipe container yang tidak ada di daftar ukuran (20/40 Feet):")
        for t in sorted(list(skipped_container_types)):
            print(f" - {t}")
    else:
        print("\n[CONTAINER] Semua tipe container berhasil dikenali.")
        
    print("="*60 + "\n")

    # 1. Standardisasi
    df_raw['Bulan'] = df_raw['Bulan'].map(month_map)
    df_raw['Ukuran'] = df_raw['Type'].map(size_map)
    df_raw = df_raw.dropna(subset=['Ukuran'])

    # 2. Agregasi
    df_agg = df_raw.groupby(['Cabang', 'Status', 'Bulan', 'Ukuran'])['Count'].sum().reset_index()

    # 3. Perkalian Khusus FAC & MAS
    mask_double = df_agg['Status'].isin(['FAC', 'MAS'])
    df_agg.loc[mask_double, 'Count'] *= 2

    # --- PEMBUATAN FILE EXCEL ---
    print(f"Membuat file output: {output_path}")
    
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        workbook = writer.book
        
        # Styles
        header_format = workbook.add_format({
            'bold': True, 'align': 'center', 'valign': 'vcenter', 'border': 1, 'bg_color': '#D9E1F2'
        })
        cell_format = workbook.add_format({'border': 1})
        total_format = workbook.add_format({'bold': True, 'border': 1, 'bg_color': '#FFF2CC'}) # Kuning muda untuk total
        
        # Proses setiap cabang
        for branch in processing_branch_list:
            branch_data = df_agg[df_agg['Cabang'] == branch]
            
            # Setup Pivot Table
            if branch_data.empty:
                df_pivot = pd.DataFrame(index=month_order)
            else:
                df_pivot = branch_data.pivot_table(
                    index='Bulan', 
                    columns=['Status', 'Ukuran'], 
                    values='Count', 
                    aggfunc='sum'
                )
                df_pivot = df_pivot.reindex(month_order)
            
            # Sort Columns
            sorted_cols = []
            for status in target_statuses:
                for size in ['20 Feet', '40 Feet']:
                    if (status, size) in df_pivot.columns:
                        sorted_cols.append((status, size))
            
            df_final = df_pivot.reindex(columns=sorted_cols)
            
            # --- HITUNG TOTAL ---
            # 1. Total Baris (Per Bulan) - di kanan
            df_final[('Total', '')] = df_final.sum(axis=1)
            
            # 2. Total Kolom (Per Status) - di bawah
            # Buat baris total
            row_total = df_final.sum(axis=0)
            row_total.name = 'Grand Total'
            df_final = pd.concat([df_final, row_total.to_frame().T])
            
            # --- MENULIS KE SHEET ---
            sheet_name = branch[:31]
            worksheet = workbook.add_worksheet(sheet_name)
            
            # Metadata Cabang
            worksheet.write(1, 1, "Cabang", header_format)
            worksheet.write(1, 2, branch, header_format)
            
            # Tambahkan Singkatan Cabang di Kolom D (Index 3)
            abbr = full_to_abbr.get(branch, "-")
            worksheet.write(1, 3, abbr, header_format)
            
            # Headers
            worksheet.write(2, 0, "Bulan", header_format)
            worksheet.write(2, 1, "Status Container", header_format)
            worksheet.write(3, 1, "Ukuran Container", header_format)
            
            # Tulis Index (Bulan + Grand Total)
            for i, idx_val in enumerate(df_final.index):
                fmt = total_format if idx_val == 'Grand Total' else cell_format
                worksheet.write(4+i, 0, idx_val, fmt)
            
            # Tulis Data Kolom
            col_idx = 1
            # df_final columns sekarang termasuk ('Total', '')
            for col_tuple in df_final.columns:
                status_lbl, size_lbl = col_tuple
                
                # Tentukan format header untuk kolom Total
                is_total_col = (status_lbl == 'Total')
                h_fmt = total_format if is_total_col else header_format
                
                # Tulis Header
                worksheet.write(2, col_idx, status_lbl, h_fmt)
                worksheet.write(3, col_idx, size_lbl, h_fmt)
                
                # Tulis Nilai
                vals = df_final[col_tuple].values
                for row_idx, val in enumerate(vals):
                    # Cek apakah ini sel Total (baris terakhir atau kolom terakhir)
                    is_total_row = (row_idx == len(vals) - 1)
                    c_fmt = total_format if (is_total_col or is_total_row) else cell_format
                    
                    if pd.notna(val) and val != 0:
                        worksheet.write(4+row_idx, col_idx, val, c_fmt)
                    else:
                        # Jika 0 di baris/kolom total tetap ditampilkan agar jelas, kalau data biasa kosong
                        if is_total_col or is_total_row:
                             worksheet.write(4+row_idx, col_idx, 0, c_fmt)
                        else:
                             worksheet.write(4+row_idx, col_idx, "", c_fmt)
                
                col_idx += 1
            
    print("File berhasil dibuat.")

Membaca file Excel...
Memproses blok data...

LAPORAN DATA YANG TIDAK MASUK PERHITUNGAN (DIABAIKAN)

[CABANG] Semua cabang berhasil dikenali.

[CONTAINER] Semua tipe container berhasil dikenali.

Membuat file output: Summary_Cabang_Alat_Berat_2025_v2.xlsx
Selesai! File berhasil dibuat.


In [1]:
import pandas as pd
import numpy as np

# --- KONFIGURASI ---
input_path = '2025 data cabang alat berat.xlsx'
output_path = 'Summary_Cabang_Alat_Berat_Split_Status.xlsx'

# 1. Daftar Nama Cabang Lengkap
full_branch_names = [
    "AMBON", "ARAR", "BAU BAU", "BIAK", "BITUNG", "BENGKULU", "BUNGKU", "BATULICIN", "BELAWAN", 
    "BANJARMASIN", "BOEPINANG", "BALIKPAPAN", "BERAU", "BANDA ACEH", "BATAM", "BUATAN", "DOBO", 
    "FAK FAK", "GORONTALO", "JAKARTA", "JAYAPURA", "KAIMANA", "KENDARI", "KETAPANG", "KUALA TANJUNG", 
    "MERAUKE", "MAKASSAR", "MANOKWARI", "NABIRE", "NUNUKAN", "PALU", "PADANG", "PALEMBANG", 
    "PONTIANAK", "PERAWANG", "SURABAYA", "SAMARINDA", "SEMARANG", "SAMPIT", "SORONG", "SERUI", 
    "TANGKIANG", "TIMIKA", "TARAKAN", "TERNATE", "TUAL"
]
additional_branches = ["MERAK", "MARUNI", "OKI"]
processing_branch_list = full_branch_names + additional_branches

# 2. Daftar Status Dasar (Urutan Display)
base_status_order = ['FXD', 'FAC', 'STR', 'MTA', 'MAS', 'MTB', 'FTL', 'FOB', 'MXD', 'MTL', 'MOB', 'FIT', 'MIT']

# 3. Mapping Ukuran Container
size_map = {
    '20 CO': '20 Feet', '20 DC': '20 Feet', '20 RH': '20 Feet', '20 RM': '20 Feet', '21 DC': '20 Feet',
    '20 FT': '20 Feet', '20 HC': '20 Feet', '20 IT': '20 Feet', '20 OT': '20 Feet', '20 RF': '20 Feet',
    '10 CO': '20 Feet', '10 DC': '20 Feet',
    '40 FT': '40 Feet', '40 HC': '40 Feet', '40 RH': '40 Feet', '40 RM': '40 Feet',
    '40 CO': '40 Feet', '40 DC': '40 Feet', '40 IT': '40 Feet', '40 OT': '40 Feet', '40 RF': '40 Feet',
    '41 DC': '40 Feet', '45 HC': '40 Feet'
}

# 4. Mapping Bulan
month_map = {
    'Jan': 'Januari', 'Feb': 'Februari', 'Mar': 'Maret', 'Apr': 'April', 'May': 'Mei', 'Jun': 'Juni',
    'Jul': 'Juli', 'Aug': 'Agustus', 'Sep': 'September', 'Oct': 'Oktober', 'Nov': 'November', 'Dec': 'Desember'
}
month_order = list(month_map.values())

# --- SCANNING STATUS & DUPLIKASI ---
print("Scanning struktur status dalam file...")
df_full = pd.read_excel(input_path, header=None)

currstate_rows = df_full[df_full[0] == 'CURRSTATE'].index.tolist()
row_labels_rows = df_full[df_full[0] == 'Row Labels'].index.tolist()

# Hitung kemunculan setiap CURRSTATE
status_counts = {}
block_info = [] # Simpan info (curr, prev) urut sesuai file

for idx in currstate_rows:
    curr = df_full.iloc[idx, 1]
    # PREV_STATE diasumsikan ada di baris tepat setelah CURRSTATE
    prev = df_full.iloc[idx + 1, 1]
    
    if curr in base_status_order:
        status_counts[curr] = status_counts.get(curr, 0) + 1
        block_info.append({
            'start_row': idx,
            'curr': curr,
            'prev': prev
        })

# Buat Mapping Nama Tampilan (Display Name)
# Aturan: Jika count > 1 -> "CURR (PREV)", Jika count == 1 -> "CURR"
display_name_map = {} # Key: (start_row), Value: Display Name
final_column_order = [] # Untuk urutan kolom nanti

# Kita butuh mengurutkan final columns berdasarkan base_status_order, 
# tapi memecah yang duplikat sesuai urutan kemunculan di file
for base in base_status_order:
    # Ambil semua blok yang cocok dengan base status ini
    blocks = [b for b in block_info if b['curr'] == base]
    
    for b in blocks:
        if status_counts[base] > 1:
            name = f"{b['curr']} ({b['prev']})"
        else:
            name = b['curr']
            
        display_name_map[b['start_row']] = name
        final_column_order.append(name)

print("Status Mapping Terbentuk:")
for k, v in display_name_map.items():
    print(f"Row {k}: {v}")

# --- PROSES DATA ---

# Setup Branch Map
if not row_labels_rows:
    raise ValueError("Tidak ditemukan baris 'Row Labels'")
first_header = row_labels_rows[0]
block1_branches = df_full.iloc[first_header+1 : first_header+1+46, 0].tolist()
branch_map = dict(zip(block1_branches, full_branch_names))
manual_map = {'MRK': 'MERAK', 'MRN': 'MARUNI', 'OKI': 'OKI'}
branch_map.update(manual_map)
full_to_abbr = {v: k for k, v in branch_map.items()}

all_data = []

print("\nMemproses data...")
for block in block_info:
    start_idx = block['start_row']
    # Ambil nama status yang sudah ditentukan tadi
    status_display = display_name_map[start_idx]
    
    # Cari header
    header_idx = next((r for r in row_labels_rows if r > start_idx), None)
    if header_idx is None: continue
    
    # Range Data
    # Cari start_idx berikutnya untuk batas bawah
    next_starts = [r for r in currstate_rows if r > start_idx]
    next_start = next_starts[0] if next_starts else len(df_full)
    
    # Metadata Kolom
    ownership_row = df_full.iloc[header_idx - 2]
    month_row = df_full.iloc[header_idx - 1]
    type_row = df_full.iloc[header_idx]
    
    col_map = {}
    curr_own = None
    curr_mon = None
    
    for c in range(1, df_full.shape[1]):
        if pd.notna(ownership_row[c]): curr_own = ownership_row[c]
        if pd.notna(month_row[c]): curr_mon = month_row[c]
        ctype = type_row[c]
        if pd.notna(ctype):
            col_map[c] = {'ownership': curr_own, 'month': curr_mon, 'type': ctype}
            
    # Loop Baris Data
    for r in range(header_idx + 1, next_start):
        row_label = df_full.iloc[r, 0]
        if pd.isna(row_label) or str(row_label).lower() == 'grand total': continue
        
        branch_name = branch_map.get(row_label, row_label if row_label in processing_branch_list else None)
        if not branch_name: continue
            
        for c, meta in col_map.items():
            val = df_full.iloc[r, c]
            if pd.notna(val) and val != 0:
                all_data.append({
                    'Cabang': branch_name,
                    'Status': status_display, # Pakai nama baru yang sudah dipisah
                    'Raw_Status': block['curr'], # Untuk cek FAC/MAS nanti
                    'Bulan': meta['month'],
                    'Type': meta['type'],
                    'Count': val
                })

if not all_data:
    print("Tidak ada data.")
else:
    df_raw = pd.DataFrame(all_data)
    df_raw['Bulan'] = df_raw['Bulan'].map(month_map)
    df_raw['Ukuran'] = df_raw['Type'].map(size_map)
    df_raw = df_raw.dropna(subset=['Ukuran'])

    # Agregasi
    # Group by Status (yang sudah unik/terpisah)
    df_agg = df_raw.groupby(['Cabang', 'Status', 'Raw_Status', 'Bulan', 'Ukuran'])['Count'].sum().reset_index()

    # Kali 2 untuk FAC/MAS (Cek Raw_Status nya)
    mask_double = df_agg['Raw_Status'].isin(['FAC', 'MAS'])
    df_agg.loc[mask_double, 'Count'] *= 2
    
    # Hapus Raw_Status agar tidak mengganggu pivot
    df_agg = df_agg.drop(columns=['Raw_Status'])

    # --- EXCEL WRITING ---
    print(f"Menulis file output: {output_path}")
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        workbook = writer.book
        header_format = workbook.add_format({'bold': True, 'align': 'center', 'valign': 'vcenter', 'border': 1, 'bg_color': '#D9E1F2'})
        cell_format = workbook.add_format({'border': 1})
        total_format = workbook.add_format({'bold': True, 'border': 1, 'bg_color': '#FFF2CC'})
        
        for branch in processing_branch_list:
            branch_data = df_agg[df_agg['Cabang'] == branch]
            
            if branch_data.empty:
                df_pivot = pd.DataFrame(index=month_order)
            else:
                df_pivot = branch_data.pivot_table(index='Bulan', columns=['Status', 'Ukuran'], values='Count', aggfunc='sum')
                df_pivot = df_pivot.reindex(month_order)
            
            # Urutkan Kolom (Pakai urutan final_column_order)
            sorted_cols = []
            for status in final_column_order:
                for size in ['20 Feet', '40 Feet']:
                    if (status, size) in df_pivot.columns:
                        sorted_cols.append((status, size))
            
            df_final = df_pivot.reindex(columns=sorted_cols)
            
            # Totals
            df_final[('Total', '')] = df_final.sum(axis=1)
            row_total = df_final.sum(axis=0)
            row_total.name = 'Grand Total'
            df_final = pd.concat([df_final, row_total.to_frame().T])
            
            # Write
            sheet_name = branch[:31]
            worksheet = workbook.add_worksheet(sheet_name)
            
            worksheet.write(1, 1, "Cabang", header_format)
            worksheet.write(1, 2, branch, header_format)
            worksheet.write(1, 3, full_to_abbr.get(branch, "-"), header_format)
            
            worksheet.write(2, 0, "Bulan", header_format)
            worksheet.write(2, 1, "Status Container", header_format)
            worksheet.write(3, 1, "Ukuran Container", header_format)
            
            for i, idx_val in enumerate(df_final.index):
                fmt = total_format if idx_val == 'Grand Total' else cell_format
                worksheet.write(4+i, 0, idx_val, fmt)
            
            col_idx = 1
            for col_tuple in df_final.columns:
                status_lbl, size_lbl = col_tuple
                is_total_col = (status_lbl == 'Total')
                h_fmt = total_format if is_total_col else header_format
                
                worksheet.write(2, col_idx, status_lbl, h_fmt)
                worksheet.write(3, col_idx, size_lbl, h_fmt)
                
                vals = df_final[col_tuple].values
                for row_idx, val in enumerate(vals):
                    is_total_row = (row_idx == len(vals) - 1)
                    c_fmt = total_format if (is_total_col or is_total_row) else cell_format
                    
                    if pd.notna(val) and val != 0:
                        worksheet.write(4+row_idx, col_idx, val, c_fmt)
                    else:
                        if is_total_col or is_total_row:
                             worksheet.write(4+row_idx, col_idx, 0, c_fmt)
                        else:
                             worksheet.write(4+row_idx, col_idx, "", c_fmt)
                col_idx += 1

    print("Selesai! File berhasil dibuat.")

Scanning struktur status dalam file...
Status Mapping Terbentuk:
Row 0: FXD (All)
Row 821: FXD (FXD)
Row 60: FAC
Row 115: STR
Row 172: MTA (FAC)
Row 228: MTA (STR)
Row 648: MTA (MXD)
Row 852: MTA (MTA)
Row 289: MAS
Row 349: MTB
Row 409: FTL (MAS)
Row 469: FTL (MTB)
Row 891: FTL (FTL)
Row 1019: FTL (FAC)
Row 529: FOB
Row 592: MXD
Row 699: MTL (MTA)
Row 927: MTL (MTL)
Row 756: MOB
Row 963: FIT
Row 991: MIT

Memproses data...
Menulis file output: Summary_Cabang_Alat_Berat_Split_Status.xlsx
Selesai! File berhasil dibuat.
